In [74]:
#libraries
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori, association_rules

In [75]:
#prepping the data
path = 'alzheimers_disease_data.csv'
data = pd.read_csv(path)
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid


In [76]:
# cleaning duplicates in data set
data.drop_duplicates(subset='PatientID', inplace=True)
data.drop(columns='PatientID', axis=1, inplace=True)
print(data.columns)

Index(['Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI', 'Smoking',
       'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality',
       'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes',
       'Depression', 'HeadInjury', 'Hypertension', 'SystolicBP', 'DiastolicBP',
       'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL',
       'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment',
       'MemoryComplaints', 'BehavioralProblems', 'ADL', 'Confusion',
       'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
       'Forgetfulness', 'Diagnosis', 'DoctorInCharge'],
      dtype='object')


In [77]:
print("Column containing missing values:")
print(data.columns[data.isnull().any()].tolist())

print("\n\nNull values in data set:")
print(data.isnull().sum())

df = pd.DataFrame(data)

Column containing missing values:
[]


Null values in data set:
Age                          0
Gender                       0
Ethnicity                    0
EducationLevel               0
BMI                          0
Smoking                      0
AlcoholConsumption           0
PhysicalActivity             0
DietQuality                  0
SleepQuality                 0
FamilyHistoryAlzheimers      0
CardiovascularDisease        0
Diabetes                     0
Depression                   0
HeadInjury                   0
Hypertension                 0
SystolicBP                   0
DiastolicBP                  0
CholesterolTotal             0
CholesterolLDL               0
CholesterolHDL               0
CholesterolTriglycerides     0
MMSE                         0
FunctionalAssessment         0
MemoryComplaints             0
BehavioralProblems           0
ADL                          0
Confusion                    0
Disorientation               0
PersonalityChanges           0
Diffic

In [78]:
# Medical History Columns

medical_history_columns = [
    'FamilyHistoryAlzheimers',
    'CardiovascularDisease',
    'Diabetes',
    'Depression',
    'HeadInjury',
    'Hypertension',
    'Diagnosis'
]


mh_columns = df[medical_history_columns].copy()

# saving this to a new excel file
output_file = "csv/medical_history_columns.csv"
mh_columns.to_csv(output_file, index=False)

# Adding new columns for the 0 values
# ex. FamilyHistoryAlzheimers will have another column with NoFamilyHistoryAlzheimers as the name
# The values for the new column will be the opposite of the old column

for col in mh_columns.columns:
    mh_columns[f'No{col}'] = mh_columns[col].apply(lambda x: 1 if x == 0 else 0)

# renaming old columns to have a Yes as the first substring

mh_columns.rename(columns={col: f'Yes{col}' for col in mh_columns.columns if not col.startswith('No')}, inplace=True)

print(mh_columns)




      YesFamilyHistoryAlzheimers  YesCardiovascularDisease  YesDiabetes  \
0                              0                         0            1   
1                              0                         0            0   
2                              1                         0            0   
3                              0                         0            0   
4                              0                         0            0   
...                          ...                       ...          ...   
2144                           0                         0            0   
2145                           0                         0            0   
2146                           0                         0            0   
2147                           0                         1            0   
2148                           0                         0            0   

      YesDepression  YesHeadInjury  YesHypertension  YesDiagnosis  \
0                 1           

In [79]:
# Medical History Not Diagnosed
mh_no = mh_columns[mh_columns['YesDiagnosis'] == 0]
mh_no = mh_no.astype(bool)

# print(mh_no)
mh_no = mh_no.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

mh_no.head()

# Apply apriori
mh_no_frequent_itemsets = apriori(mh_no, min_support=0.5, use_colnames=True)


# Generate rules
mh_no_rules = association_rules(mh_no_frequent_itemsets, metric='lift', min_threshold=1.01, num_itemsets=len(mh_no_frequent_itemsets))
mh_no_rules = mh_no_rules.sort_values(by='lift', ascending=False)
mh_no_rules = mh_no_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])
# mh_no_rules = mh_no_rules[mh_no_rules['consequents'].apply(lambda x: 'NoDiagnosis' in x and len(x) == 1)]
# mh_no_rules = mh_no_rules.replace([np.inf, -np.inf], np.nan).dropna()

print(mh_no_frequent_itemsets)
print(mh_no_rules)

# mh_columns = mh_columns.astype(bool)

# mh_columns_frequent_itemsets = apriori(mh_columns, min_support=0.5, use_colnames=True)
# print(mh_columns_frequent_itemsets)

output_file_assoc_rules = "csv/medical_history_NotDiagnosed.csv"
mh_no_rules.to_csv(output_file_assoc_rules, index=False)


     support                                           itemsets
0   0.737221                        (NoFamilyHistoryAlzheimers)
1   0.863931                          (NoCardiovascularDisease)
2   0.840893                                       (NoDiabetes)
3   0.797696                                     (NoDepression)
4   0.902808                                     (NoHeadInjury)
5   0.860331                                   (NoHypertension)
6   0.637869  (NoFamilyHistoryAlzheimers, NoCardiovascularDi...
7   0.614111            (NoFamilyHistoryAlzheimers, NoDiabetes)
8   0.591073          (NoFamilyHistoryAlzheimers, NoDepression)
9   0.665947          (NoFamilyHistoryAlzheimers, NoHeadInjury)
10  0.639309        (NoFamilyHistoryAlzheimers, NoHypertension)
11  0.719942              (NoCardiovascularDisease, NoDiabetes)
12  0.688265            (NoCardiovascularDisease, NoDepression)
13  0.784017            (NoCardiovascularDisease, NoHeadInjury)
14  0.741541          (NoCardiovascularD

In [80]:
# Medical History Not Diagnosed
mh_yes = mh_columns[mh_columns['YesDiagnosis'] == 1]
mh_yes = mh_yes.astype(bool)

# print(mh_no)
mh_yes = mh_yes.drop(columns=['YesDiagnosis', 'NoDiagnosis'])
mh_yes.head()

# Apply apriori
mh_yes_frequent_itemsets = apriori(mh_yes, min_support=0.5, use_colnames=True)


# Generate rules
mh_yes_rules = association_rules(mh_yes_frequent_itemsets, metric='lift', min_threshold=1.03, num_itemsets=len(mh_yes_frequent_itemsets))
mh_yes_rules = mh_yes_rules.sort_values(by='lift', ascending=False)
mh_yes_rules = mh_yes_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# mh_yes_rules = mh_yes_rules[mh_yes_rules['consequents'].apply(lambda x: 'YesDiagnosis' in x and len(x) == 1)]
# mh_yes_rules = mh_yes_rules.replace([np.inf, -np.inf], np.nan).dropna()

print(mh_yes_frequent_itemsets)
print(mh_yes_rules)

output_file_assoc_rules = "csv/medical_history_YesDiagnosed.csv"
mh_yes_rules.to_csv(output_file_assoc_rules, index=False)


     support                                           itemsets
0   0.767105                        (NoFamilyHistoryAlzheimers)
1   0.840789                          (NoCardiovascularDisease)
2   0.864474                                       (NoDiabetes)
3   0.802632                                     (NoDepression)
4   0.915789                                     (NoHeadInjury)
5   0.834211                                   (NoHypertension)
6   0.652632  (NoFamilyHistoryAlzheimers, NoCardiovascularDi...
7   0.663158            (NoFamilyHistoryAlzheimers, NoDiabetes)
8   0.610526          (NoFamilyHistoryAlzheimers, NoDepression)
9   0.694737          (NoFamilyHistoryAlzheimers, NoHeadInjury)
10  0.636842        (NoFamilyHistoryAlzheimers, NoHypertension)
11  0.735526              (NoCardiovascularDisease, NoDiabetes)
12  0.678947            (NoCardiovascularDisease, NoDepression)
13  0.763158            (NoCardiovascularDisease, NoHeadInjury)
14  0.700000          (NoCardiovascularD

In [81]:
# All medical history columns

mh_columns = mh_columns.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

mh_columns = mh_columns.astype(bool)
mh_columns.head()

# Apply apriori
mh_columns_frequent_itemsets = apriori(mh_columns, min_support=0.3, use_colnames=True)

# Generate rules
mh_columns_rules = association_rules(mh_columns_frequent_itemsets, metric='lift', min_threshold=1.03, num_itemsets=len(mh_columns_frequent_itemsets))
mh_columns_rules = mh_columns_rules.sort_values(by='lift', ascending=False)
mh_columns_rules = mh_columns_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# mh_columns_rules = mh_columns_rules[mh_columns_rules['consequents'].apply(lambda x: ('NoDiagnosis' in x or 'YesDiagnosis' in x) and len(x) == 1)]

print(mh_columns_frequent_itemsets)
print(mh_columns_rules)

output_file_assoc_rules = "csv/medical_history_All.csv"
mh_columns_rules.to_csv(output_file_assoc_rules, index=False)

     support                                           itemsets
0   0.747790                        (NoFamilyHistoryAlzheimers)
1   0.855747                          (NoCardiovascularDisease)
2   0.849232                                       (NoDiabetes)
3   0.799442                                     (NoDepression)
4   0.907399                                     (NoHeadInjury)
..       ...                                                ...
58  0.419265  (NoCardiovascularDisease, NoHeadInjury, NoDiab...
59  0.402047  (NoCardiovascularDisease, NoDepression, NoHead...
60  0.393206  (NoDepression, NoHeadInjury, NoDiabetes, NoHyp...
61  0.449511  (NoCardiovascularDisease, NoDepression, NoHead...
62  0.341554  (NoCardiovascularDisease, NoDepression, NoHead...

[63 rows x 2 columns]
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage]
Index: []


In [82]:
# Symptoms 
symptom_and_assessment_columns = [
  # 'MemoryComplaints',
  # 'BehavioralProblems',
  'Confusion',
  'Disorientation',
  'PersonalityChanges',
  'DifficultyCompletingTasks',
  'Forgetfulness',
  'Diagnosis'
]

s_columns = df[symptom_and_assessment_columns].copy()


# saving this to a new excel file
output_file = "csv/symptom_and_assessment_columns.csv"
s_columns.to_csv(output_file, index=False)

# Adding new columns for the 0 values
# ex. FamilyHistoryAlzheimers will have another column with NoFamilyHistoryAlzheimers as the name
# The values for the new column will be the opposite of the old column

for col in s_columns.columns:
    s_columns[f'No{col}'] = s_columns[col].apply(lambda x: 1 if x == 0 else 0)

# renaming old columns to have a Yes as the first substring

s_columns.rename(columns={col: f'Yes{col}' for col in s_columns.columns if not col.startswith('No')}, inplace=True)

print(s_columns)


      YesConfusion  YesDisorientation  YesPersonalityChanges  \
0                0                  0                      0   
1                0                  0                      0   
2                0                  1                      0   
3                0                  0                      0   
4                0                  0                      1   
...            ...                ...                    ...   
2144             1                  0                      0   
2145             0                  0                      0   
2146             0                  0                      0   
2147             0                  0                      0   
2148             0                  1                      0   

      YesDifficultyCompletingTasks  YesForgetfulness  YesDiagnosis  \
0                                1                 0             0   
1                                0                 1             0   
2                    

In [83]:
# Symptoms Not Diagnosed
s_no = s_columns[s_columns['YesDiagnosis'] == 0]
s_no = s_no.astype(bool)

s_no = s_no.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

s_no.head()

# Apply apriori
s_no_frequent_itemsets = apriori(s_no, min_support=0.3, use_colnames=True)

# Generate rules

s_no_rules = association_rules(s_no_frequent_itemsets, metric='lift', min_threshold=1.02, num_itemsets=len(s_no_frequent_itemsets))
s_no_rules = s_no_rules.sort_values(by='lift',ascending=False)
s_no_rules = s_no_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# s_no_rules = s_no_rules[s_no_rules['confidence'] > .80]

print(s_no_frequent_itemsets)
print(s_no_rules)

output_file_assoc_rules = "csv/symptom_NotDiagnosed.csv"
s_no_rules.to_csv(output_file_assoc_rules, index=False)

     support                                           itemsets
0   0.301656                                 (YesForgetfulness)
1   0.789057                                      (NoConfusion)
2   0.835133                                 (NoDisorientation)
3   0.843772                             (NoPersonalityChanges)
4   0.843772                      (NoDifficultyCompletingTasks)
5   0.698344                                  (NoForgetfulness)
6   0.660187                    (NoDisorientation, NoConfusion)
7   0.664507                (NoPersonalityChanges, NoConfusion)
8   0.668107         (NoDifficultyCompletingTasks, NoConfusion)
9   0.555076                     (NoForgetfulness, NoConfusion)
10  0.699784           (NoPersonalityChanges, NoDisorientation)
11  0.705544    (NoDifficultyCompletingTasks, NoDisorientation)
12  0.579554                (NoDisorientation, NoForgetfulness)
13  0.716343  (NoPersonalityChanges, NoDifficultyCompletingT...
14  0.589633            (NoPersonalityCh

In [84]:
# Symtoms Diagnosed

s_yes = s_columns[s_columns['YesDiagnosis'] == 1]
s_yes = s_yes.astype(bool)

s_yes = s_yes.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

s_yes.head()

# Apply apriori
s_yes_frequent_itemsets = apriori(s_yes, min_support=0.5, use_colnames=True)

# Generate rules
s_yes_rules = association_rules(s_yes_frequent_itemsets, metric='lift', min_threshold=1.01, num_itemsets=len(s_yes_frequent_itemsets))
s_yes_rules = s_yes_rules.sort_values(by='lift', ascending=False)
s_yes_rules = s_yes_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# print(s_yes_frequent_itemsets)
print(s_yes_rules)

output_file_assoc_rules = "csv/symptom_YesDiagnosed.csv"
s_yes_rules.to_csv(output_file_assoc_rules, index=False)

                                       antecedents  \
0       (NoDifficultyCompletingTasks, NoConfusion)   
1                           (NoPersonalityChanges)   
2  (NoDifficultyCompletingTasks, NoDisorientation)   
3                           (NoPersonalityChanges)   

                                       consequents  antecedent support  \
0                           (NoPersonalityChanges)            0.660526   
1       (NoDifficultyCompletingTasks, NoConfusion)            0.859211   
2                           (NoPersonalityChanges)            0.705263   
3  (NoDifficultyCompletingTasks, NoDisorientation)            0.859211   

   consequent support   support  confidence      lift  leverage  
0            0.859211  0.577632    0.874502  1.017797  0.010100  
1            0.660526  0.577632    0.672282  1.017797  0.010100  
2            0.859211  0.613158    0.869403  1.011863  0.007188  
3            0.705263  0.613158    0.713629  1.011863  0.007188  


In [85]:
# Symptoms All
s_columns = s_columns.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

s_columns = s_columns.astype(bool)
s_columns.head()

#  Apply apriori
s_columns_frequent_itemsets = apriori(s_columns, min_support=0.1, use_colnames=True)

# Generate rules
s_columns_rules = association_rules(s_columns_frequent_itemsets, metric='lift', min_threshold=1.05, num_itemsets=len(s_columns_frequent_itemsets))
s_columns_rules = s_columns_rules[s_columns_rules['confidence'] > 0.8]
s_columns_rules = s_columns_rules.sort_values(by='lift', ascending=False)
s_columns_rules = s_columns_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])


print(s_columns_frequent_itemsets)
print(s_columns_rules)

output_file_assoc_rules = "csv/symptom_All.csv"
s_columns_rules.to_csv(output_file_assoc_rules, index=False)


     support                                           itemsets
0   0.205212                                     (YesConfusion)
1   0.158213                                (YesDisorientation)
2   0.150768                            (YesPersonalityChanges)
3   0.158678                     (YesDifficultyCompletingTasks)
4   0.301536                                 (YesForgetfulness)
..       ...                                                ...
81  0.381573  (NoDifficultyCompletingTasks, NoForgetfulness,...
82  0.399721  (NoPersonalityChanges, NoDifficultyCompletingT...
83  0.412285  (NoPersonalityChanges, NoDifficultyCompletingT...
84  0.151698  (YesForgetfulness, NoDifficultyCompletingTasks...
85  0.328060  (NoDifficultyCompletingTasks, NoPersonalityCha...

[86 rows x 2 columns]
                                          antecedents         consequents  \
6   (YesForgetfulness, NoDifficultyCompletingTasks...  (NoDisorientation)   
12  (YesForgetfulness, NoDifficultyCompletingTasks...  

In [86]:
yesno_columns = [
  'FamilyHistoryAlzheimers',
  'CardiovascularDisease',
  'Diabetes',
  'Depression',
  'HeadInjury',
  'Hypertension',
  'MemoryComplaints',
  'BehavioralProblems',
  'Confusion',
  'Disorientation',
  'PersonalityChanges',
  'DifficultyCompletingTasks',
  'Forgetfulness',
  'Diagnosis'
]

yn_columns = df[yesno_columns].copy()


# saving this to a new excel file

output_file_assoc_rules = "csv/alzheimers_categorical_columns.csv"
yn_columns.to_csv(output_file_assoc_rules, index=False)

# Adding new columns for the 0 values

for col in yn_columns.columns:
  yn_columns[f'No{col}'] = yn_columns[col].apply(lambda x: 1 if x == 0 else 0)

# renaming old columns to have a Yes as the first substring

yn_columns.rename(columns={col: f'Yes{col}' for col in yn_columns.columns if not col.startswith('No')}, inplace=True)

print(yn_columns)


      YesFamilyHistoryAlzheimers  YesCardiovascularDisease  YesDiabetes  \
0                              0                         0            1   
1                              0                         0            0   
2                              1                         0            0   
3                              0                         0            0   
4                              0                         0            0   
...                          ...                       ...          ...   
2144                           0                         0            0   
2145                           0                         0            0   
2146                           0                         0            0   
2147                           0                         1            0   
2148                           0                         0            0   

      YesDepression  YesHeadInjury  YesHypertension  YesMemoryComplaints  \
0                 1    

In [87]:
# Categorical Columns Not Diagnosed
yn_no = yn_columns[yn_columns['YesDiagnosis'] == 0]
yn_no = yn_no.astype(bool)

yn_no = yn_no.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

yn_no.head()

# Apply apriori
yn_no_frequent_itemsets = apriori(yn_no, min_support=0.1, use_colnames=True)

# Generate rules
yn_no_rules = association_rules(yn_no_frequent_itemsets, metric='lift', min_threshold=1.2, num_itemsets=len(yn_no_frequent_itemsets))
yn_no_rules = yn_no_rules.sort_values(by='lift', ascending=False)
yn_no_rules = yn_no_rules[yn_no_rules['confidence'] > 0.7]
yn_no_rules = yn_no_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# yn_no_rules = yn_no_rules[yn_no_rules['consequents'].apply(lambda x: 'NoDiagnosis' in x and len(x) == 1)]

# print(yn_no_frequent_itemsets)
print(yn_no_rules)

output_file_assoc_rules = "csv/alzheimers_categorical_NotDiagnosed.csv"
yn_no_rules.to_csv(output_file_assoc_rules, index=False)

                                          antecedents  \
9   (YesForgetfulness, NoDifficultyCompletingTasks...   
16  (YesForgetfulness, NoDifficultyCompletingTasks...   
0   (YesForgetfulness, NoDifficultyCompletingTasks...   

                                          consequents  antecedent support  \
9   (NoMemoryComplaints, NoCardiovascularDisease, ...            0.131030   
16  (NoMemoryComplaints, NoCardiovascularDisease, ...            0.145428   
0   (NoMemoryComplaints, NoCardiovascularDisease, ...            0.145428   

    consequent support   support  confidence      lift  leverage  
9             0.645068  0.103672    0.791209  1.226550  0.019149  
16            0.584593  0.103672    0.712871  1.219431  0.018655  
0             0.645068  0.113751    0.782178  1.212551  0.019940  


In [88]:
# Categorical columns Diagnosed
yn_yes = yn_columns[yn_columns['YesDiagnosis'] == 1]
yn_yes = yn_yes.astype(bool)

yn_yes = yn_yes.drop(columns=['YesDiagnosis', 'NoDiagnosis'])

yn_yes.head()

# Apply apriori

yn_yes_frequent_itemsets = apriori(yn_yes, min_support=0.1, use_colnames=True)

# Generate rules

yn_yes_rules = association_rules(yn_yes_frequent_itemsets, metric='lift', min_threshold=1.3, num_itemsets=len(yn_yes_frequent_itemsets))
yn_yes_rules = yn_yes_rules.sort_values(by='lift', ascending=False)
yn_yes_rules = yn_yes_rules[yn_yes_rules['confidence'] > 0.6]
yn_yes_rules = yn_yes_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# yn_yes_rules = yn_yes_rules[yn_yes_rules['consequents'].apply(lambda x: 'YesDiagnosis' in x and len(x) == 1)]

# print(yn_yes_frequent_itemsets)
print(yn_yes_rules)

output_file_assoc_rules = "csv/alzheimers_categorical_YesDiagnosed.csv"

yn_yes_rules.to_csv(output_file_assoc_rules, index=False)


                                           antecedents  \
122  (YesForgetfulness, NoDepression, NoHeadInjury,...   
42   (NoDifficultyCompletingTasks, NoCardiovascular...   
132  (YesForgetfulness, NoPersonalityChanges, NoDep...   
124  (YesForgetfulness, NoDepression, NoDiabetes, N...   
133  (YesForgetfulness, NoDepression, NoHypertensio...   
81   (NoDifficultyCompletingTasks, YesBehavioralPro...   
57   (NoDifficultyCompletingTasks, NoCardiovascular...   
83   (NoDifficultyCompletingTasks, YesBehavioralPro...   
9    (NoDifficultyCompletingTasks, NoCardiovascular...   
99   (NoCardiovascularDisease, NoDifficultyCompleti...   
3    (NoCardiovascularDisease, YesBehavioralProblem...   
72   (NoDifficultyCompletingTasks, YesBehavioralPro...   
123  (YesForgetfulness, NoDepression, NoHeadInjury,...   
30   (NoPersonalityChanges, NoCardiovascularDisease...   
58   (NoDifficultyCompletingTasks, NoFamilyHistoryA...   

                                           consequents  antecedent supp

In [89]:
# Categorical columns All

yn_columns = yn_columns.drop(columns=['YesDiagnosis', 'NoDiagnosis'])
yn_columns = yn_columns.astype(bool)

# Apply apriori
yn_columns_frequent_itemsets = apriori(yn_columns, min_support=0.1, use_colnames=True)

# Generate rules

yn_columns_rules = association_rules(yn_columns_frequent_itemsets, metric='lift', min_threshold=1.1, num_itemsets=len(yn_columns_frequent_itemsets))
# yn_columns_rules = yn_columns_rules[yn_columns_rules['consequents'].apply(lambda x: ('NoDiagnosis' in x or 'YesDiagnosis' in x) and len(x) == 1)]
yn_columns_rules = yn_columns_rules[yn_columns_rules['confidence']>0.8]
yn_columns_rules = yn_columns_rules.sort_values(by='lift', ascending=False)
yn_columns_rules = yn_columns_rules.drop(columns=['conviction','representativity', 'zhangs_metric','jaccard','certainty','kulczynski'])

# print(yn_columns_frequent_itemsets)
print(yn_columns_rules)

output_file_assoc_rules = "csv/alzheimers_categorical_All.csv"
yn_columns_rules.to_csv(output_file_assoc_rules, index=False)

                                           antecedents  \
358  (YesForgetfulness, NoDifficultyCompletingTasks...   
246  (YesForgetfulness, NoCardiovascularDisease, No...   
364  (YesForgetfulness, NoCardiovascularDisease, No...   
336  (YesForgetfulness, NoDiabetes, NoHypertension,...   
78   (YesForgetfulness, NoBehavioralProblems, NoHyp...   
200  (YesForgetfulness, NoHypertension, NoFamilyHis...   
324  (YesForgetfulness, NoMemoryComplaints, NoDiabe...   
68   (YesForgetfulness, NoHypertension, NoDiabetes,...   
208  (YesForgetfulness, NoCardiovascularDisease, No...   

                          consequents  antecedent support  consequent support  \
358  (NoDisorientation, NoHeadInjury)            0.135412            0.767334   
246  (NoDisorientation, NoHeadInjury)            0.137273            0.767334   
364  (NoDisorientation, NoHeadInjury)            0.121452            0.767334   
336  (NoDisorientation, NoHeadInjury)            0.140996            0.767334   
78   (NoDisori